# AAI614: Data Science & its Applications

*Notebook 6.4: Track this disease*

<a href="https://colab.research.google.com/github/harmanani/AAI614/blob/main/Week%206/Notebook6.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
np.random.seed(0)
mean = 0
variance = 2

# Modeling dart coordinates using two Normal distributions
x = np.random.normal(mean, variance ** 0.5)
y = np.random.normal(mean, variance ** 0.5)
print(f"The x coordinate of a randomly thrown dart is {x:.2f}")
print(f"The y coordinate of a randomly thrown dart is {y:.2f}")

**Simulating randomly thrown darts**

In [ ]:
import matplotlib.pyplot as plt
np.random.seed(1)
bulls_eye1 = [0, 0]
bulls_eye2 = [6, 0]
bulls_eyes = [bulls_eye1, bulls_eye2]
x_coordinates, y_coordinates = [], []
for bulls_eye in bulls_eyes:
    for _ in range(5000):
        x = np.random.normal(bulls_eye[0], variance ** 0.5)
        y = np.random.normal(bulls_eye[1], variance ** 0.5)
        x_coordinates.append(x)
        y_coordinates.append(y)
        
plt.scatter(x_coordinates, y_coordinates)
plt.show()

**Assigning darts to the nearest bull’s-eye**

In [ ]:
from scipy.spatial.distance import euclidean
def nearest_bulls_eye(dart):
    distances = [euclidean(dart, bulls_e) for bulls_e in bulls_eyes]
    return np.argmin(distances)

darts = [[0,1], [6, 1]]
for dart in darts:
    index = nearest_bulls_eye(dart)
    print(f"The dart at position {dart} is closest to bulls-eye {index}")

Now, we will apply the `nearest_bulls_eye` function to all our computed color coordinates. Afterwards, each dart-point will be plotted using one of two colors, in order to distinguish between the two bull's-eye assignments.

**Coloring darts based on nearest bull’s-eye**

In [ ]:
def color_by_cluster(darts):
    nearest_bulls_eyes = [nearest_bulls_eye(dart) for dart in darts]
    for bs_index in range(len(bulls_eyes)):
        selected_darts = [darts[i] for i in range(len(darts))
                          if bs_index == nearest_bulls_eyes[i]]
        x_coordinates, y_coordinates = np.array(selected_darts).T
        plt.scatter(x_coordinates, y_coordinates, 
                    color=['g', 'k'][bs_index])
    plt.show()

darts = [[x_coordinates[i], y_coordinates[i]]  
         for i in range(len(x_coordinates))]
color_by_cluster(darts)

The colored darts sensibly split into two even clusters. How would we identify such clusters if no central coordinates were provided? Well, one primitive strategy is to simply guess the location of the bull's-eyes.

**Assigning darts to randomly chosen centers**

In [ ]:
bulls_eyes = np.array(darts[:2])
color_by_cluster(darts)

Cluster B on the right seems to be stretching way too far to the left.  Lets remedy our error. We 'll compute the mean coordinates of all the points within the stretched right clustered group, and afterwards utilize these coordinates to adjust our estimation of the group's center. We will also reset the left-most cluster's center to its mean prior to re-running our centrality-based clustering.

**Assigning darts to centers based on mean**

In [ ]:
def update_bulls_eyes(darts):
    updated_bulls_eyes = []
    nearest_bulls_eyes = [nearest_bulls_eye(dart) for dart in darts]
    for bs_index in range(len(bulls_eyes)):
        selected_darts = [darts[i] for i in range(len(darts))
                          if bs_index == nearest_bulls_eyes[i]]
        x_coordinates, y_coordinates = np.array(selected_darts).T
        mean_center = [np.mean(x_coordinates), np.mean(y_coordinates)]
        updated_bulls_eyes.append(mean_center)
        
    return updated_bulls_eyes

bulls_eyes = update_bulls_eyes(darts)
color_by_cluster(darts)

The cluster's centers still appear a little off. Lets remedy the results by repeating the mean-based centrality adjustment over 10 additional iterations.

**Adjusting bull’s-eye positions over 10 iterations**

In [ ]:
for i in range(10):
    bulls_eyes = update_bulls_eyes(darts)
    
color_by_cluster(darts)

Now the two sets of darts have been perfectly clustered. We have essentially replicated the  **K-means** clustering algorithm, which organizes data using centrality.

## K-Means: A Clustering Algorithm for Grouping Data into K Central Groups

### K-means Clustering Using Scikit-Learn

A speedy implementation of the K-means algorithm is available through the external Scikit-Learn library. Lets import Scikit-learn's `KMeans` clustering class.

Use the `KMeans` class to cluster our `darts` data.

**K-means clustering using Scikit-learn**

In [ ]:
cluster_model = KMeans(n_clusters=2)
assigned_bulls_eyes = cluster_model.fit_predict(darts)

print("Bull's-eye assignments:")
print(assigned_bulls_eyes)

Lets quickly color our darts based on their clustering assignments, in order to confirm that the assignments makes sense.

**Plotting K-means cluster assignments**

In [ ]:
for bs_index in range(len(bulls_eyes)):
    selected_darts = [darts[i] for i in range(len(darts))
                      if bs_index == assigned_bulls_eyes[i]]
    x_coordinates, y_coordinates = np.array(selected_darts).T
    plt.scatter(x_coordinates, y_coordinates, 
                color=['g', 'k'][bs_index])
plt.show()

Our clustering model has located the centroids in the data. Now, we can reuse these centroids to analyze new data-points that the model has not seen before.

**Using `cluster_model` to cluster new data**

In [ ]:
new_darts = [[500, 500], [-500, -500]]
new_bulls_eye_assignments = cluster_model.predict(new_darts)
for i, dart in enumerate(new_darts):
    bulls_eye_index = new_bulls_eye_assignments[i]
    print(f"Dart at {dart} is closest to bull's-eye {bulls_eye_index}")

### Selecting the Optimal K Using the Elbow Method

We estimate an appropriate value for K using a technique known as the **Elbow method**. The Elbow method depends on a calculated value called **inertia**, which is the sum of the squared distances between each point and its closest K-means center. We'll run the technique by plotting the inertia of our dartboard dataset over a large range of K values.

**Plotting the K-means inertia**

In [ ]:
k_values = range(1, 10)
inertia_values = [KMeans(k).fit(darts).inertia_
                  for k in k_values]

plt.plot(k_values, inertia_values)
plt.xlabel('K')
plt.ylabel('Inertia')
plt.show()

The generated plot resembles an arm bent at the elbow. The elbow points directly to a K of 2. What will happen if the number of centers is increased? We can find out by adding an additional bull's-eye to our dart-throwing simulation.

**Plotting inertia for a 3-dartboard simulation**

In [ ]:
new_bulls_eye = [12, 0]
for _ in range(5000):
    x = np.random.normal(new_bulls_eye[0], variance ** 0.5)
    y = np.random.normal(new_bulls_eye[1], variance ** 0.5)
    darts.append([x, y])

inertia_values = [KMeans(k).fit(darts).inertia_
                  for k in k_values]

plt.plot(k_values, inertia_values)
plt.xlabel('K')
plt.ylabel('Inertia')
plt.show()

## Using Density to Discover Clusters

Suppose that an astronomer discovers a new planet at the far-flung edges of the solar system. The planet, much like our Saturn, has multiple rings spinning in constant orbit around its center. Each ring is formed from thousands of rocks. We'll model these rocks as individual points, defined by x and y coordinates.

**Simulating rings around a planet**

In [ ]:
from sklearn.datasets import make_circles

x_coordinates = []
y_coordinates = []
for factor in [.3, .6, 0.99]:
    rock_ring, _ = make_circles(n_samples=800, factor=factor,
                                noise=.03, random_state=1)
    for rock in rock_ring:
        x_coordinates.append(rock[0])
        y_coordinates.append(rock[1])

plt.scatter(x_coordinates, y_coordinates)
plt.show()

Three ring-groups are clearly present in the plot. Lets search for these three clusters using K-means.

**Using K-means to cluster rings**

In [ ]:
rocks = [[x_coordinates[i], y_coordinates[i]]  
          for i in range(len(x_coordinates))]
rock_clusters = KMeans(3).fit_predict(rocks)

colors = [['g', 'y', 'k'][cluster] for cluster in  rock_clusters]
plt.scatter(x_coordinates, y_coordinates, color=colors)
plt.show()

The output is an utter failure! We need to design an algorithm that will cluster data within dense regions of space. One simple definition of density is as follows; a point is in a dense region only if it's located within a distance `epsilon` of `min_points` other points.  Below, we'll set `epsilon` to 0.1 and `min_points` to 10.

**Specifying density parameters**

In [18]:
epsilon = 0.1
min_points = 10

Lets analyze the density of the first rock in our `rocks` list. We'll begin by searching for all the other rocks that are within `epsilon` units of `rocks[0]`.

**Finding the neighbors of `rocks[0]`**

In [19]:
neighbor_indices = [i for i, rock in enumerate(rocks[1:])
                    if euclidean(rocks[0], rock) <= epsilon]

Now, we'll compare the number of neighbors to `min_points`, in order to determine if `rocks[0]` lies in a dense region of space.

**Checking the density of `rocks[0]`**

In [ ]:
num_neighbors = len(neighbor_indices)
print(f"The rock at index 0 has {num_neighbors} neighbors.")

if num_neighbors >= min_points:
    print("It lies in a dense region.")
else:
    print("It does not lie in a dense region.")

The rock at index 0 lies in a dense region of space. We can combine `rocks[0]` and its neighbors into a single dense cluster.

**Creating a dense cluster**

In [ ]:
dense_region_indices = [0] + neighbor_indices
dense_region_cluster = [rocks[i] for i in dense_region_indices]
dense_cluster_size = len(dense_region_cluster)
print(f"We found a dense cluster containing {dense_cluster_size} rocks")

The rock and index 0 and its neighbors form a single 41-element dense cluster. What about the neighbors of the neighbors? By analyzing additional neighboring points, we expand the size of `dense_region_cluster`.

**Expanding a dense cluster**

In [ ]:
dense_region_indices = set(dense_region_indices)
for index in neighbor_indices:
    point = rocks[index]
    neighbors_of_neighbors = [i for i, rock in enumerate(rocks)
                              if euclidean(point, rock) <= epsilon]
    if len(neighbors_of_neighbors) >= min_points:
        dense_region_indices.update(neighbors_of_neighbors)
            
dense_region_cluster = [rocks[i] for i in dense_region_indices]
dense_cluster_size = len(dense_region_cluster)
print(f"We expanded our cluster to include {dense_cluster_size} rocks")

We can expand our cluster even further by analyzing the density of newly encountered neighbors. Iteratively repeating our analysis will increase the breadth of our cluster boundary. This precedure is known as **DBSCAN**. The DBSCAN algorithm organizes data based on its density distribution.

## DBSCAN: A Clustering Algorithm for Grouping Data Based on Spatial Density

Scikit-Learn makes DBSCAN available for use. We simply need to import the `DBSCAN` class from `sklearn.cluster`. Afterwards, we can initialize the class by assigning `epsilon` and `min_points` using the `eps` and `min_samples` parameters. Lets utilize `DBSCAN` to cluster our 3 rings.

**Using `DBSCAN` to cluster rings**

In [ ]:
from sklearn.cluster import DBSCAN
cluster_model = DBSCAN(eps=epsilon, min_samples=min_points)
rock_clusters = cluster_model.fit_predict(rocks)
colors = [['g', 'y', 'k'][cluster] for cluster in rock_clusters]
plt.scatter(x_coordinates, y_coordinates, color=colors)
plt.show()

DBSCAN has successfully identified the 3 rock rings. The algorithm succeeded where K-means had failed.

### Comparing DBSCAN and K-means

DBSCAN can filter random outliers located in sparse regions of space. For example, if we add an outlier located beyond the boundary of the rings, then DBSCAN will assign it a cluster id of -1. The negative value indicates that the outlier cannot be clustered with the rest of the dataset.

**Finding outliers using DBSCAN**

In [24]:
noisy_data = rocks + [[1000, -1000]]
clusters = DBSCAN(eps=epsilon, 
                  min_samples=min_points).fit_predict(noisy_data)
assert clusters[-1] == -1

There is one other advantage to the DBSCAN technique that is missing from K-means. DBSCAN does not depend on Euclidean distance.

### Clustering Based on Non-Euclidean Distance

Suppose we are visiting Manhattan. We wish to know the walking distance from the Empire State Building to Columbus Circle. The Empire State Building is located at the intersection of 34th street and 5th avenue. Meanwhile, Columbus Circle is located at the intersection of 57th street and 8th avenue. Our route requires us to walk 26 blocks total. Manhattan's average block-length is 0.17 miles. Lets compute that walking distance directly using a generalized `manhattan_distance` function.

**Computing the Manhattan distance**

In [ ]:
def manhattan_distance(point_a, point_b):
    num_blocks = np.sum(np.absolute(point_a - point_b))
    return 0.17 * num_blocks

x = np.array([34, 5])
y = np.array([57, 8])
distance = manhattan_distance(x, y)

print(f"Manhattan distance is {distance} miles")

Now, suppose we wish to cluster more than two Manhattan locations, using DBSCAN. We will pass `metric= manhattan_distance` into the initialization method. Consequently, the clustering distance will correctly reflect the grid-based constraints within the City.

**Clustering using Manhattan distance**

In [ ]:
points = [[35, 5], [33, 6], [37, 4], [40, 7], [45, 5]]
clusters = DBSCAN(eps=1, min_samples=3,
                  metric=manhattan_distance).fit_predict(points)

for i, cluster in enumerate(clusters):
    point = points[i]
    if cluster == -1:
        print(f"Point at index {i} is an outlier")
        plt.scatter(point[0], point[1], marker='x', color='k')
    else:
        print(f"Point at index {i} is in cluster {cluster}")
        plt.scatter(point[0], point[1], color='g')

plt.grid(True, which='both', alpha=0.5)
plt.minorticks_on()

plt.show()

Unlike K-means, the DBSCAN algorithm does not require our distance function to be linearly divisible. Thus, we can easily run DBSCAN clustering using a ridiculous distance metric.

**Clustering using a ridiculous measure of distance**

In [ ]:
def ridiculous_measure(point_a, point_b):
    is_negative_a = np.array(point_a) < 0
    is_negative_b = np.array(point_b) < 0
    if is_negative_a.all() and is_negative_b.all():
        return 0
    elif is_negative_a.any() or is_negative_b.any():
        return 10
    else:
        return 2

points = [[-1, -1], [-10, -10], [-1000, -13435], [3,5], [5,-7]]
                   
clusters = DBSCAN(eps=.1, min_samples=2,
                  metric=ridiculous_measure).fit_predict(points)

for i, cluster in enumerate(clusters):
    point = points[i]
    if cluster == -1:
        print(f"{point} is an outlier")
    else:
        print(f"{point} falls in cluster {cluster}")

## Analyzing Clusters Using Pandas

We can more intuitively analyze clustered rocks by combining the coordinates and the clusters together in a single Pandas table.

**Storing clustered coordinates in a table**

In [35]:
import pandas as pd
x_coordinates, y_coordinates = np.array(rocks).T
df = pd.DataFrame({'X': x_coordinates, 'Y': y_coordinates,
                   'Cluster': rock_clusters})

Our Pandas table lets us easily access the rocks in any cluster. Lets plot those rocks that fall into cluster zero.

**Plotting a single cluster using Pandas**

In [ ]:
df_cluster = df[df.Cluster == 0]
plt.scatter(df_cluster.X, df_cluster.Y)
plt.show()

Pandas allows us to obtain a table containing elements from any single cluster. Alternatively, we might want to obtain multiple tables, where each table maps to a cluster id. In Pandas, this can easily be done by calling `df.groupby('Cluster')`.

**Iterating over clusters using Pandas**

In [ ]:
for cluster_id, df_cluster in df.groupby('Cluster'):
    if cluster_id == 0:
        print(f"Skipping over cluster {cluster_id}")
        continue
    
    print(f"Plotting cluster {cluster_id}")
    plt.scatter(df_cluster.X, df_cluster.Y)

plt.show()

# II. Geographic Location Visualization and Analysis

## The Great-Circle Distance: A Metric for Computing Distances Between Two Global Points

The direct path between two points along the surface of a sphere is called the **great-circle distance**. That distance  depends on a series of well-known trigonometric operations.

**Defining a great-circle distance function**

In [39]:
from math import cos, sin, asin

def great_circle_distance(x1, y1, x2, y2):
    delta_x, delta_y = x2 - x1, y2 - y1
    haversin = sin(delta_x / 2) ** 2 + np.product([cos(x1), cos(x2), 
                                                   sin(delta_y / 2) ** 2])
    return 2 * asin(haversin ** 0.5)

Lets calculate the great-circle distance between two points that lie 180 degrees apart, relative to both the x-axis and the y-axis.

**Computing the great-circle distance**

In [ ]:
from math import pi
distance = great_circle_distance(0, 0, 0, pi)
print(f"The distance equals {distance} units")

The points are exactly π units apart, half the distance required to circumnavigate a unit-circle. This is akin to traveling between the North and South Poles of any planet. We'll confirm by analyzing the latitudes and longitudes of Earth's North Pole and South Pole. Lets begin by recording the known coordinates of each pole.

In [48]:
latitude_north, longitude_north = (90.0, 0)
latitude_south, longitude_south = (-90.0, 0)

Latitudes and longitudes measure spherical coordinates in degrees, not radians. We'll thus convert to radians from degrees using the `np.radians` function. Afterwards, we'll input the radian results into `great_circle_distance`.

**Computing the great-circle distance between poles**

In [ ]:
to_radians =  np.radians([latitude_north, longitude_north, 
                          latitude_south, longitude_south])
distance = great_circle_distance(*to_radians.tolist())
print(f"The unit-circle distance between poles equals {distance} units")

As expected, the distance between poles on a unit-sphere is π . Now, let's measure the distance between two poles here on Earth. The radius of the Earth is not 1 hypothetical unit, but rather 3956 actual miles.

**Computing the travel distance between Earth’s poles**

In [ ]:
earth_distance = 3956 * distance
print(f"The distance between poles equals {earth_distance} miles")

Lets create a general `travel_distance` function to calculate the travel mileage between any two terrestrial points.

**Defining a travel distance function**

In [52]:
def travel_distance(lat1, lon1, lat2, lon2):
    to_radians =  np.radians([lat1, lon1, lat2, lon2])
    return 3956 * great_circle_distance(*to_radians.tolist())

assert travel_distance(90, 0, -90, 0) == earth_distance

## Plotting Maps Using Cartopy

Cartopy is a Matplotlib extension for generating maps in Python. Lets install and import the Cartopy library.

**Importing the Cartopy library**

In [35]:
import cartopy

A geographic map is a 2D representation of a 3D surface on a globe. Flattening the spherical globe is carried out using a process called **projection**. The simplest projection involves superimposing the globe on an unrolled cylinder, and is called the **Plate Carrée projection**. We'll utilize this standard projection in our plots by importing `PateCarree` from `cartopy.csr`.

**Importing the Plate Carrée projection**

In [37]:
from cartopy.crs import PlateCarree

The `PlateCarree` class can be used in conjunction with Matploblib in order to visualize the Earth. For instance, running `plt.axes(projection=PlateCarree()).coastlines()` will plot the outlines of the Earth's seven continents.

**Visualizing the Earth using Cartopy**

In [ ]:
plt.axes(projection=PlateCarree()).coastlines()
plt.show()

Our plotted map is a bit small. We can increase map-size using Matplotlib's `plt.figure` function.

**Visualizing a larger map of the Earth**

In [ ]:
plt.figure(figsize=(12, 8))
plt.axes(projection=PlateCarree()).coastlines()
plt.show()

So far our map looks sparse and uninviting. We can improve the quality by calling `plt.axes(projection=PlateCarree()).stock_img()`. The method-call will color the map using topographic information.

**Coloring a map of the Earth**

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.axes(projection=PlateCarree()).stock_img()
plt.show()

National boundaries are currently missing from the plot. Let's add them in:

In [ ]:
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=PlateCarree())
ax.coastlines()
ax.stock_img()
plt.show()

Please note that `ax.stock_img()` relies on a saved stock image of the Earth to color the map. Alternatively, we can color the oceans and the continents using the `ax.add_features` method. The method displays special Cartopy features, which are stored in the `cartopy.feature` module.

**Adding colors with the `features` module**

In [ ]:
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=PlateCarree())
ax.coastlines()
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND)
plt.show()

Currently, national borders are currently missing from the plot.
Cartopy treats these borders as a feature in the `features` module. We can incorporate country borders by calling `ax.add_feature(cartopy.feature.BORDERS)`.

**Adding national borders to the plot**

In [ ]:
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=PlateCarree())
ax.coastlines()
ax.add_feature(cartopy.feature.BORDERS)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND)
plt.show()

Suppose we are given a list of locations defined by pairs of latitudes and longitudes. We can plot these locations on our map by separating the latitudes from the longitudes and then passing the results into `ax.scatter`.

**Plotting coordinates on a map**

In [ ]:
plt.figure(figsize=(12, 8))
coordinates = [(39.9526, -75.1652), (37.7749, -122.4194),
               (40.4406, -79.9959), (38.6807, -108.9769),
               (37.8716, -112.2727), (40.7831, -73.9712)]

latitudes, longitudes = np.array(coordinates).T
ax = plt.axes(projection=PlateCarree())
ax.scatter(longitudes, latitudes)
ax.set_global()
ax.coastlines()
plt.show()

The plotted points all fall within the borders of North America. We can thusly simplify the map by zooming-in on that continent. However, first we’ll need to adjust the **map extent**, which is the geographic area shown on a map.

**Plotting North American Coordinates**

In [ ]:
plt.figure(figsize=(12, 8))
ax = plt.axes(projection=PlateCarree())

north_america_extent = (-145, -50, 0, 90)
ax.set_extent(north_america_extent)
ax.scatter(longitudes, latitudes, color='r')

def add_map_features():
    ax.coastlines()
    ax.add_feature(cartopy.feature.BORDERS)
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAND)

add_map_features()
plt.show()

We successfully zoomed in on North America. Now, we'll zoom in further, onto the United States. Unfortunately, the Plate Carrée projection will prove insufficient for this purpose. That technique distorts the map if we zoom-in too close to any country. Instead, we will on rely on the **Lambert Conformal Conic projection**.

**Plotting USA Coordinates**

In [ ]:
from cartopy.crs import LambertConformal

plt.figure(figsize=(12, 8))
ax = plt.axes(projection=LambertConformal())
us_extent = (-120, -75, 20, 50)
ax.set_extent(us_extent)

ax.scatter(longitudes, latitudes, color='r', 
           transform=PlateCarree(),
           s=100)

add_map_features()
plt.show()

Our map of the United States is looking a little sparse. Lets add state borders to the map by calling ``ax.add_feature(cartopy.feature.STATES)`.

**Plotting a US map with state border inclusion**

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=LambertConformal())
ax.set_extent(us_extent)

ax.scatter(longitudes, latitudes, color='r', 
           transform=PlateCarree(),
           s=100)

ax.add_feature(cartopy.feature.STATES)
add_map_features()
plt.show()

Basemap allows us to plot any location on a map. All we need is the location’s latitude and longitude. Thus, we need a mapping between location names and their geographic properties. That mapping is provided by the GeoNamesCache location-tracking library.

## Location Tracking Using GeoNamesCache

GeoNamesCache is designed to efficiently retrieve data pertaining to continents, countries, and cities, as well US counties and US states. Lets install the library and explore its usage in more detail. We'll begin by initializing a `GeonamesCache` location-tracking object.

**Initializing a `GeonamesCache` object**

In [ ]:
from geonamescache import GeonamesCache
gc = GeonamesCache()

Lets use our `gc` object to explore the 7 continents. We'll run `gc.get_continents()` in order to retrieve a dictionary of continent-related information.

**Fetching all seven continents from GeoNamesCache**

In [ ]:
continents = gc.get_continents()
print(continents.keys())

The dictionary keys represent shorthand encoding of continent names, in which __Africa__ is transformed into `'AF'`, and __North America__ is transformed into `'NA'`. Lets check the values mapped to every key by passing in the code for __North America__.

**Fetching North America from GeoNamesCache**

In [ ]:
north_america = continents['NA']
print(north_america.keys())

Many of the `north_america` data elements represent the various naming schemes for the North American continent. Such information is not very useful.

**Printing North America’s naming schemas**

In [ ]:
for name_key in ['name', 'asciiName', 'toponymName']:
    print(north_america[name_key])

The  `'lat'` and the `'lng'` keys map to the latitude and longitude of the central-most location in the continent. We can utilize these coordinates to plot a map projection centered at the heart of North America.

**Mapping North America’s central coordinates**

In [ ]:
latitude = float(north_america['lat'])
longitude = float(north_america['lng'])

plt.figure(figsize=(12, 8))
ax = plt.axes(projection=PlateCarree())
ax.set_extent(north_america_extent)
ax.scatter([longitude], [latitude], s=200)
add_map_features()
plt.show()

## Accessing Country Information
We can analyze countries using the `get_countries` method. It returns a dictionary whose 2-character keys encode the names of 252 different countries. Accessing `gc.get_countries()['US']` will return a dictionary containing useful USA statistics. Lets output all the non-city information pertaining to the United States.

**Fetching US data from GeoNamesCache**

In [ ]:
countries = gc.get_countries()
num_countries = len(countries)
print(f"GeonamesCache holds data for {num_countries} countries.")

us_data = countries['US']
print("The following data pertains to the United States:")
print(us_data)

There is valuable information within each country's `'neighbours'` element. It maps to a comma-delimited string
of country codes that signify all neighboring territories. We can obtain more details about each neighbor by splitting the string and passing the codes into the `'countries'` dictionary.

**Fetching neighboring countries**

In [ ]:
us_neighbors = us_data['neighbours']
for neighbor_code in us_neighbors.split(','):
    print(countries[neighbor_code]['name'])

We can also query all countries by name using the `get_countries_by_names` method. This method returns a dictionary whose elements are country names rather than codes.

**Fetching countries by name**

In [ ]:
result = gc.get_countries_by_names()['United States']
assert result == countries['US']
countries['US']

## Accessing City Information

The `get_cities` method returns a dictionary whose keys are unique ids mapping back to city data.

**Fetching cities from GeoNamesCache**

In [ ]:
cities = gc.get_cities()
num_cities = len(cities)
print(f"GeoNamesCache holds data for {num_cities} total cities")
city_id = list(cities.keys())[0]
print(cities[city_id])

The data for each city contains the reference code for the country where that city is located. By utilizing the country code, we can create a new mapping between a country and all its territorial cities.

**Fetching US cities from GeoNamesCache**

In [ ]:
us_cities = [city for city in cities.values() 
             if city['countrycode'] == 'US']
num_us_cities = len(us_cities)
print(f"GeoNamesCache holds data for {num_us_cities} US cities.")

Lets find the average US latitude and longitude. This average will approximate the central coordinates of the United States.

**Approximating US central coordinates**

In [ ]:
center_lat = np.mean([city['latitude'] 
                      for city in us_cities])
center_lon = np.mean([city['longitude'] 
                       for city in us_cities])

fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=LambertConformal())
ax.set_extent(us_extent)
ax.scatter([center_lon], [center_lat], transform=PlateCarree(), s=200)
ax.add_feature(cartopy.feature.STATES)
add_map_features()
plt.show()

The `get_cities` method is suitable for iterating over city information, but not for querying cities by name. To run a name-based city search, we must rely on `get_cities_by_name`. 

**Fetching cities by name**

In [ ]:
matched_cities_list = gc.get_cities_by_name('Philadelphia')
print(matched_cities_list)

The `get_cities_by_name` method may return more than one city, because city-names are not always unique. For example, GeoNamesCache contains 6 different instances of the city __San Francisco__, spanning across 5 different countries.

**Fetching multiple cities with a shared name**

In [ ]:
matched_cities_list = gc.get_cities_by_name('San Francisco')

for i, san_francisco in enumerate(matched_cities_list):
    city_info = list(san_francisco.values())[0]
    country_code = city_info['countrycode']
    country = countries[country_code]['name']
    print(f"The San Francisco at index {i} is located in {country}")

Its common for multiple cities to share the same name. Choosing among such cities is quite difficult. Usually, the safest guess is the city with the largest population. 

**Mapping the most populous San Francisco**

In [ ]:
best_sf = max(gc.get_cities_by_name('San Francisco'), 
              key=lambda x: list(x.values())[0]['population'])
sf_data = list(best_sf.values())[0]
sf_lat = sf_data['latitude']
sf_lon = sf_data['longitude']

plt.figure(figsize=(12, 8))
ax = plt.axes(projection=LambertConformal())
ax.set_extent(us_extent)
ax.scatter(sf_lon, sf_lat, transform=PlateCarree(), s=200)
add_map_features()
ax.text(sf_lon + 1, sf_lat, ' San Francisco', fontsize=16, transform=PlateCarree())
plt.show()

### Limitations of the GeoNamesCache Library

The `get_cities_by_name` method maps only one version of a city's name to its geographic data. This poses a problem for cities like __New York__, which carry more than one commonly referenced name.

**Fetching New York City from GeoNamesCache**

In [ ]:
for ny_name in ['New York', 'New York City']:
    if not gc.get_cities_by_name(ny_name):
        print(f"'{ny_name}' is not present in GeoNamesCache database.")
    else:
        print(f"'{ny_name}' is present in GeoNamesCache database.")

The limits of single references become particularly obvious when we examine diacritics in city names. Diacritics are accent marks that designate the proper pronunciation of foreign-sounding words. 

**Fetching accented cities from GeoNamesCache**

In [ ]:
print(gc.get_cities_by_name(u'Cañon City'))
print(gc.get_cities_by_name(u'Hagåtña'))

How many of the cities stored in GeoNamesCache contain diacritics in their name? We can find out using the `unidecode` function from the external Unidecode library.

**Counting all accented cities in GeoNamesCache**

In [ ]:
from unidecode import unidecode
accented_names = [city['name'] for city in gc.get_cities().values()
                  if city['name'] != unidecode(city['name'])]
num_accented_cities = len(accented_names)

print(f"An example accented city name is '{accented_names[0]}'")
print(f"{num_accented_cities} cities have accented names") 

We can now match the stripped dictionary keys against all inputted text by passing the accented dictionary values into GeoNamesCache, whenever a key-match is found.

**Finding accent-free city-names in text**

In [ ]:
alternative_names = {unidecode(name): name 
                     for name in accented_names}
print(gc.get_cities_by_name(alternative_names['Hagatna']))

## Matching Location Names in Text

In Python, we can easily determine if one string is a substring of another, or if the start of a string contains some predefined text.

**Basic string matching**

In [ ]:
text = u'This sentence matches Hagatna'
for key, value in alternative_names.items():
    if key in text:
        print(gc.get_cities_by_name(value))
        break

In more complex analyses, Python's basic string syntax can be quite limiting. For example, Python's string methods can't directly distinguish between sub-characters in a string and sub-phrases in a sentence.

**Basic substring matching errors**

In [77]:
assert 'in a' in 'sin apple'
assert 'in a' in 'win attached'

To overcame these limitations, we must rely on Python's built-in regular expression processing library, `re`. A **regular expression** (or **regex** for short) is a string-encoded pattern that can be compared against some text. Most regex-matching in Python can be executed with the `re.search` function. 

**String matching using regexes**

In [78]:
import re
regex = 'Boston'
random_text = 'Clown Patty'
match = re.search(regex, random_text)
assert match is None

matchable_text = 'Boston Marathon'
match = re.search(regex, matchable_text)
assert match is not None
start, end = match.start(), match.end()
matched_string = matchable_text[start: end]
assert matched_string == 'Boston'

Case-insensitive string matching is a breeze with `re.search`. We simply pass `re.IGNORECASE` as an added `flags` parameter.

**Case-insensitive matching using regexes**

In [79]:
for text in ['BOSTON', 'boston', 'BoSTOn']:
    assert re.search(regex, text, flags=re.IGNORECASE) is not None

Also, regexes permit us to match exact words, and not just substrings, using word boundary detection.

**Word boundary matching using regexes**

In [80]:
for regex in ['\\bin a\\b', r'\bin a\b']:
    for text in ['sin apple', 'win attached']:
        assert re.search(regex, text) is None
        
    text = 'Match in a string'
    assert re.search(regex, text) is not None

Now, let us carry out a more complicated match. We'll match against the sentence `f'I visited {city} yesterday`, where `{city}` represents one of 3 possible locations; `'Boston'`, `'Philadelphia'`, or `'San Francisco'`.

**Multi-city matching using regexes**

In [81]:
regex = r'I visited \b(Boston|Philadelphia|San Francisco)\b yesterday.'
assert re.search(regex, 'I visited Chicago yesterday.') is None

cities = ['Boston', 'Philadelphia', 'San Francisco']
for city in cities:
    assert re.search(regex, f'I visited {city} yesterday.') is not None

Suppose we want to match a regex against 100 strings. For every match, `re.search`  will transform the regex into Python `PatternObject.` Each such transformation is computationally costly. We're better off executing the transformation only once using `re.compile`.

**String matching using compiled regexes**

In [82]:
compiled_re = re.compile(regex)
text = 'I visited Boston yesterday.'
for i in range(1000):
    assert compiled_re.search(text) is not None

# 12. Case Study: Track this Disease by extracting Locations from Headline Data

We'll begin by loading the headline data.

**Loading headline data**

In [ ]:
headline_file = open('headlines.txt','r')
headlines = [line.strip() 
             for line in headline_file.readlines()]
num_headlines = len(headlines)
print(f"{num_headlines} headlines have been loaded")

We need a mechanism for extracting city and country names from the headline text. One naïve solution is to match the locations in GeoNamesCache against each and every headline.  However, for more optimal matching, we should transform each location name into a case-independent and accent-independent regular expression. Lets execute these transformations using a custom `name_to_regex` function. 

**Converting names to regexes**

In [10]:
def name_to_regex(name):
    decoded_name = unidecode(name)
    if name != decoded_name:
        regex = fr'\b({name}|{decoded_name})\b'
    else:
        regex = fr'\b{name}\b'
    return re.compile(regex, flags=re.IGNORECASE)

Using `name_to_regex`, we can create create a mapping between regular expressions and the original names in GeoNamesCache.

**Mapping names to regexe**

In [ ]:
countries = [country['name'] 
             for country in gc.get_countries().values()]
country_to_name = {name_to_regex(name): name 
                   for name in countries}
                   
cities = [city['name'] for city in gc.get_cities().values()]
city_to_name = {name_to_regex(name): name for name in cities}

Next, we’ll use our mappings to define a function that will look for location names in text

**Finding locations in text**

In [86]:
def get_name_in_text(text, dictionary):
    for regex, name in sorted(dictionary.items(), 
                              key=lambda x: x[1]):
        if regex.search(text):
            return name
    return None

We'll utilize `get_names_in_text` to discover the cities and countries that are mentioned in the `headlines` list. Afterwards, we'll store the results in a Pandas table for easier analysis.

**Finding locations in headlines**

In [87]:
import pandas as pd

matched_countries = [get_name_in_text(headline, country_to_name)
                     for headline in headlines]
matched_cities = [get_name_in_text(headline, city_to_name)
                  for headline in headlines]
data = {'Headline': headlines, 'City': matched_cities, 
        'Country': matched_countries}
df = pd.DataFrame(data)

Lets explore our location table. We'll start by summarizing the contents of `df` using the `describe` method.

**Summarizing the location data**

In [ ]:
summary = df[['City', 'Country']].describe()
print(summary)

The most frequently mentioned city is apparently __Of, Turkey__. The 45 instances of __Of__ are more likely
to match the preposition than the rarely referenced Turkish location. We will output some instances of __Of__ in order to confirm the error.

**Fetching cities named __Of__**

In [ ]:
of_cities = df[df.City == 'Of'][['City', 'Headline']]
ten_of_cities = of_cities.head(10)
print(ten_of_cities.to_string(index=False))

In all the wrongly matched headlines we matched to __Of__ but not to the actual city
name. The mismatches occurred because we didn't consider potential multiple matches in a headline. How frequently do headlines contain two or more city matches? Lets find out. 

**Finding multi-city headlines**

In [ ]:
def get_cities_in_headline(headline):
    cities_in_headline = set()
    for regex, name in city_to_name.items():          
        match = regex.search(headline)
        if match:
            if headline[match.start()].isupper():
                cities_in_headline.add(name)
                
    return list(cities_in_headline)

df['Cities'] = df['Headline'].apply(get_cities_in_headline)
df['Num_cities'] = df['Cities'].apply(len)
df_multiple_cities = df[df.Num_cities > 1]
num_rows, _ = df_multiple_cities.shape
print(f"{num_rows} headlines match multiple cities")

67 headlines contain more than one city, representing approximately 10% of the data. Why are so many headlines matching against multiple locations? Perhaps exploring some sample matches will yield an answer.

**Sampling multi-city headlines**

In [ ]:
ten_cities = df_multiple_cities[['Cities', 'Headline']].head(10)
print(ten_cities.to_string(index=False))

Short, invalid city names are getting matched to the headlines along with longer, more correct location names. One solution is simply to assign the longest city-name as the representative location if more than one matched city is found.

**Selecting the longest city names**

In [93]:
def get_longest_city(cities):
    if cities:
        return max(cities, key=len)
    return None

df['City'] = df['Cities'].apply(get_longest_city)

As a sanity check, we'll output those rows in the the table that contain a short city-name (4 characters or less), in order to ensure that no erroneous short name is getting assigned to one of our headlines.

**Printing the shortest city names**

In [ ]:
short_cities = df[df.City.str.len() <= 4][['City', 'Headline']]
print(short_cities.to_string(index=False))

Let's now shift our attention from cities to countries. Only 15 of the total headlines contain actual country information. The count is low enough for us to manually examine all these headlines.

**Fetching headlines with countries**

In [ ]:
df_countries = df[df.Country.notnull()][['City', 
                                         'Country', 
                                         'Headline']]
print(df_countries.to_string(index=False))

All of the country-bearing headlines also contain city information. Thus, we can assign a latitude and longitude without relying on the country's central coordinates. Consequently, we can disregard the country names from our analysis.

**Dropping countries from the table**

In [96]:
df.drop('Country', axis=1, inplace=True)

We are nearly ready to add latitudes and longitudes to our table. However, we first need to consider those rows where no locations were detected. Lets count the number of unmatched headlines, and then print a subset of that data.

**Exploring unmatched headlines**

In [ ]:
df_unmatched = df[df.City.isnull()]
num_unmatched = len(df_unmatched)
print(f"{num_unmatched} headlines contain no city matches.")
print(df_unmatched.head(10)[['Headline']].values)

Approximately 6% of the headlines did not match any cities. Given that low frequency, we will delete the missing mentions.

**Dropping unmatched headlines**

In [98]:
df = df[~df.City.isnull()][['City', 'Headline']] 

## Visualizing and Clustering the Extracted Location Data

All the rows in our table contain a city-name. Now, we can assign a latitude and longitude to each row.

**Assigning geographic coordinates to cities**

In [99]:
latitudes, longitudes = [], []
for city_name in df.City.values:
    city = max(gc.get_cities_by_name(city_name), 
              key=lambda x: list(x.values())[0]['population'])
    city = list(city.values())[0]
    latitudes.append(city['latitude']) 
    longitudes.append(city['longitude'])

df = df.assign(Latitude=latitudes, Longitude=longitudes)

Lets execute K-means across our set of 2D coordinates. We'll use the Elbow method to choose a reasonable value for K.

**Plotting a geographic elbow curve**

In [ ]:
coordinates = df[['Latitude', 'Longitude']].values
k_values = range(1, 10)
inertia_values = []
for k in k_values:
    inertia_values.append(KMeans(k).fit(coordinates).inertia_)

plt.plot(range(1, 10), inertia_values)
plt.xlabel('K')
plt.ylabel('Inertia')
plt.show()

The "elbow" within our Elbow plot points to a K of 3 That K-value is very low; limiting our scope to at-most 3 different geographic territories.

**Using K-means to cluster cities into three groups**

In [ ]:
def plot_clusters(clusters, longitudes, latitudes):
    plt.figure(figsize=(12, 10))
    ax = plt.axes(projection=PlateCarree())
    ax.coastlines()
    ax.scatter(longitudes, latitudes, c=clusters)
    ax.set_global()
    plt.show()


df['Cluster'] = KMeans(3).fit_predict(coordinates)
plot_clusters(df.Cluster, df.Longitude, df.Latitude)

These continental categories are too broad to actually be useful. Perhaps our K was too low after all. We'll disregard the recommended K-value from the Elbow analysis, and double the size of K to 6.

**Using K-means to cluster cities into six groups**

In [ ]:
df['Cluster'] = KMeans(6).fit_predict(coordinates)
plot_clusters(df.Cluster, df.Longitude, df.Latitude)

K-mean's sense of centrality is unable to properly distinguish between Africa, Europe and Asia. As an alternative approach, we can attempt to execute DBSCAN clustering. The DBSCAN algorithm takes as input any distance metric
of our choosing, allowing us to cluster on the great-circle distance between points.

**Defining a NumPy-based great-circle metric**

In [17]:
def great_circle_distance(coord1, coord2, radius=3956):
    if np.array_equal(coord1, coord2):
        return 0.0 

    coord1, coord2 = np.radians(coord1), np.radians(coord2)
    delta_x, delta_y = coord2 - coord1
    haversin = sin(delta_x / 2) ** 2 + np.product([cos(coord1[0]),
                                                   cos(coord2[0]), 
                                                   sin(delta_y / 2) ** 2])
    return  2 * radius * asin(haversin ** 0.5)

With our distance metric in place, we are ready to run the DBSCAN algorithm. 

**Using DBSCAN to cluster cities**

In [ ]:
metric = great_circle_distance
dbscan = DBSCAN(eps=250, min_samples=3, metric=metric)
df['Cluster'] = dbscan.fit_predict(coordinates)

DBSCAN assigns -1 to outlier data-points that do not cluster. Lets remove these outliers from our table. Afterwards, we’ll plot the remaining results.

**Plotting non-outlier DBSCAN clusters**

In [ ]:
df_no_outliers = df[df.Cluster > -1]
plot_clusters(df_no_outliers.Cluster, df_no_outliers.Longitude,
              df_no_outliers.Latitude)

DBSCAN has does a decent job of generating discrete clusters within parts of South America, Asia, and Southern Africa. The Eastern United States however, falls into a single overly-dense cluster. Lets cluster US locations independently from the rest of the World. To do so, we will first assign country-codes across each of our cities.

**Assigning country codes to cities**

In [124]:
def get_country_code(city_name):
    city = max(gc.get_cities_by_name(city_name), 
               key=lambda x: list(x.values())[0]['population'])
    return list(city.values())[0]['countrycode']

df['Country_code'] = df.City.apply(get_country_code)

The country-codes allow us to separate the data into two distinct `DataFrame` objects. The first object, `df_us`,  which hold all the United States locations. The second object, `df_not_us`, will hold all the remaining global cities.

**Separating US and global cities**

In [125]:
df_us = df[df.Country_code == 'US']
df_not_us = df[df.Country_code != 'US']

We've separated US and non-US cities. Now, we will need to re-cluster the coordinates within the two separated tables. 

**Re-clustering extracted cities**

In [126]:
def re_cluster(input_df, eps):
    input_coord = input_df[['Latitude', 'Longitude']].values
    dbscan = DBSCAN(eps=eps, min_samples=3, 
                    metric=great_circle_distance)
    clusters = dbscan.fit_predict(input_coord)
    input_df = input_df.assign(Cluster=clusters)
    return input_df[input_df.Cluster > -1]

df_not_us = re_cluster(df_not_us, 250)
df_us = re_cluster(df_us, 125)

## Extracting Insights from Location Clusters

Lets investigate the clustered data within the `df_not_us` table. We'll start by grouping the clustered results using the Pandas `groupby` method.

**Grouping cities by cluster**

In [ ]:
groups = df_not_us.groupby('Cluster')
num_groups = len(groups)
print(f"{num_groups} Non-US have been clusters detected")

31 global clusters have been detected. Lets sort these groups by size and count the headlines in the largest cluster.

**Finding the largest cluster**

In [ ]:
sorted_groups = sorted(groups, key=lambda x: len(x[1]), 
                       reverse=True)
group_id, largest_group = sorted_groups[0]
group_size = len(largest_group)
print(f"Largest cluster contains {group_size} headlines")

The largest cluster contains 51 total headlines. Reading all these headlines individually will be a time-consuming process. We can save time by outputting just those headlines that represent the most central locations in the cluster.

**Computing cluster centrality**

In [129]:
def compute_centrality(group):
    group_coords = group[['Latitude', 'Longitude']].values
    center = group_coords.mean(axis=0)
    distance_to_center = [great_circle_distance(center, coord)
                          for coord in group_coords]
    group['Distance_to_center'] = distance_to_center

Computing the centrality allows us to sort the grouped locations based on their distance to the centers, in order to output the most central headlines. Lets print the 5 most central headlines within our largest cluster.

**Finding the central headlines in largest cluster**

In [ ]:
def sort_by_centrality(group):
    compute_centrality(group)
    return group.sort_values('Distance_to_center', ascending=True)

largest_group = sort_by_centrality(largest_group)
for headline in largest_group.Headline.values[:5]:
    print(headline)

The central headlines in largest_cluster focus on an outbreak of Mad Cow Disease within various European cities. We can confirm that the cluster’s locale is centered in Europe by outputting the top countries associated with cities in the cluster.

**Finding the top three countries in largest cluster**

In [ ]:
from collections import Counter
def top_countries(group):
    countries = [gc.get_countries()[country_code]['name']
                 for country_code in group.Country_code.values]
    return Counter(countries).most_common(3)


print(top_countries(largest_group))

Lets repeat this analysis across the next four largest global clusters.

**Summarizing content within the largest clusters**

In [ ]:
for _, group in sorted_groups[1:5]:
    sorted_group = sort_by_centrality(group)
    print(top_countries(sorted_group))
    for headline in sorted_group.Headline.values[:5]:
        print(headline)
    print('\n')

Lets turn our attention to the US clusters. We'll start by visualizing the clusters on a map of the United States.

**Plotting United States DBSCAN clusters**

In [ ]:
plt.figure(figsize=(12, 10))
ax = plt.axes(projection=LambertConformal())
ax.set_extent(us_extent)
ax.scatter(df_us.Longitude, df_us.Latitude, c=df_us.Cluster, 
		   transform=PlateCarree())
ax.coastlines()
ax.add_feature(cartopy.feature.STATES)
plt.show()

The visualized map yields reasonable outputs. We'll proceed to analyze the top 5 US clusters by printing their centrality-sorted headlines.

**Summarizing content within the largest US clusters**

In [ ]:
us_groups = df_us.groupby('Cluster')
us_sorted_groups = sorted(us_groups, key=lambda x: len(x[1]),
                         reverse=True)
for _, group in us_sorted_groups[:5]:
    sorted_group = sort_by_centrality(group)
    for headline in sorted_group.Headline.values[:5]:
        print(headline)
    print('\n')

Lets plot one additional image. It will summarize the menacing scope of the spreading Zika epidemic. The image will display all US and global clusters where Zika is mentioned in more than 50% of article headlines.

**Plotting Zika clusters**

In [ ]:
def count_zika_mentions(headlines):
    zika_regex = re.compile(r'\bzika\b', 
                            flags=re.IGNORECASE)
    zika_count = 0
    for headline in headlines:
        if zika_regex.search(headline): 
            zika_count += 1
    
    return zika_count

fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection=PlateCarree())

for _, group in sorted_groups + us_sorted_groups:
    headlines = group.Headline.values
    zika_count = count_zika_mentions(headlines)
    if float(zika_count) / len(headlines) > 0.5:
        ax.scatter(group.Longitude, group.Latitude)

ax.coastlines()
ax.set_global()
plt.show()